In [1]:
from api import API
import pandas as pd

In [2]:
api = API()

## Load / Merge Data

In [3]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [4]:
d = api.buildings()
bldings = list(d.values())

In [7]:
@interact(blding=bldings)
def get_df(blding):
    bld = api.building(blding).id
    timestamp = '2017-08-18 00:45:00'

    # Load some data
    rooms = api.building_rooms(bld)
    points = api.building_points(bld)
    vals = api.building_values_at_time(bld, timestamp)
    
    # Display first five entries of each
    display(rooms.head(), f"len: {len(rooms)}")
    display(points.head(), f"len: {len(points)}")
    display(vals.head(), f"len: {len(vals)}")

interactive(children=(Dropdown(description='blding', options=('Boliou Hall', 'Burton Hall', 'CMC', 'Cassat Hall', 'Cassat Hall/James Hall', 'Cowling Gym', 'Davis Hall', 'Evans', 'Evans Hall', 'Faculty Club', 'Goodhue', 'Goodhue Hall', 'Goodsell', 'Gould Library', 'Hulings', 'Hulings Hall', 'James', 'James Hall', 'LDC', 'Laird Hall', 'Leighton Hall', 'Leighton Hall ', 'Libe', 'Main Campus', 'Music Hall', 'Musser Hall', 'Myers Hall', 'Nourse Hall', 'Olin Hall', 'Olin Hall ', 'Rec', 'Recreation Center', 'Sayles-Hill Campus Center', 'Scoville Hall', 'Severance Hall', 'Skinner Chapel', 'Student Townhouses', 'Watson Hall', 'Weitz Center for Creativity', 'West Gym', 'Willis Hall'), value='Boliou Hall'), Output()), _dom_classes=('widget-interact',))

In [11]:
bld = api.building('Cassat Hall/James Hall').id
timestamp = '2017-08-18 00:45:00'

# Load some data
rooms = api.building_rooms(bld)
points = api.building_points(bld)
vals = api.building_values_at_time(bld, timestamp)

display(rooms.head(), f"len: {len(rooms)}")
display(points.head(), f"len: {len(points)}")
display(vals.head(), f"len: {len(vals)}")

ValueError: Expected object or value

In [3]:
bld = api.building('Hulings').id
timestamp = '2017-08-18 00:45:00'

# Load some data
rooms = api.building_rooms(bld)
points = api.building_points(bld)
vals = api.building_values_at_time(bld, timestamp)

In [7]:
# Remove room 6, the scary dummy room
rooms = rooms[rooms['id'] != 6]

In [8]:
# Merge df together
df = pd.merge(points, rooms, left_on='roomid', right_on='id',
              suffixes=('_point', '_room'))

df= pd.merge(df, vals, left_on='id_point', right_on='pointid',)

In [10]:
df.head(n=3)

,description,id_point,name_point,pointsourceid,pointtypeid,roomid,buildingid,id_room,name_room,factor,id,name,pointid,pointname,pointtimestamp,pointvalue,returntype,units,date,time
0,112 RH VALVE,572,HU.R112.RHV,2,1,18,4,18,112,2,1,floatPSI2,572,HU.R112.RHV,2017-08-18 00:45:00,3.00,float,PSI,2017-08-18,00:45
1,112 TEMP,573,HU.R112.RM,2,13,18,4,18,112,2,13,floatDEG F2,573,HU.R112.RM,2017-08-18 00:45:00,70.92,float,DEG F,2017-08-18,00:45
2,112 TEMP,574,HU.R112.RMT,2,13,18,4,18,112,2,13,floatDEG F2,574,HU.R112.RMT,2017-08-18 00:45:00,70.92,float,DEG F,2017-08-18,00:45


## Attempt to tag specific points

In [11]:
df['tag'] = 'none'

# Tag 'valve' based on description
df_valve = df[df['description'].str.find('VALVE') > 0]
df.loc[df_valve.index, 'tag'] = 'valve'

# Tag 'temp' based on point name
df_temp = df[df['name_point'].str.find('.RM') > 0]
df.loc[df_temp.index, 'tag'] = 'temp1 (RM)'

df_temp = df[df['name_point'].str.find('.RMT') > 0]
df.loc[df_temp.index, 'tag'] = 'temp2 (RMT)'

In [12]:
# View DF where tag is not "none
view_df = df.query('tag != "none"')

# Pivot 
view_df = (view_df.pivot(index='name_room', columns='tag',
                        values='pointvalue')
           .reset_index().rename_axis(None, axis=1))

view_df

,name_room,temp1 (RM),temp2 (RMT),valve
0,102,73.39,73.39,12.0
1,112,70.92,70.92,3.0
2,121,70.61,70.61,9.0
3,202,69.60,71.01,15.0
4,203,67.65,67.65,9.0
5,206,69.10,69.10,9.0
6,207,76.93,76.93,9.0
7,212,72.18,72.18,3.0
8,213,71.60,71.60,9.0
9,215,69.11,69.11,9.0


In [14]:
for idx, row in view_df.iterrows():
    print(row.name_room)

102
112
121
202
203
206
207
212
213
215


In [13]:
view_df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>name_room</th>\n      <th>temp1 (RM)</th>\n      <th>temp2 (RMT)</th>\n      <th>valve</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>102</td>\n      <td>73.39</td>\n      <td>73.39</td>\n      <td>12.0</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>112</td>\n      <td>70.92</td>\n      <td>70.92</td>\n      <td>3.0</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>121</td>\n      <td>70.61</td>\n      <td>70.61</td>\n      <td>9.0</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>202</td>\n      <td>69.60</td>\n      <td>71.01</td>\n      <td>15.0</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>203</td>\n      <td>67.65</td>\n      <td>67.65</td>\n      <td>9.0</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>206</td>\n      <td>69.10</td>\n      <td>69.10</td>\n      <td>9.0</td>\n    </tr>\n    <tr>\n      <th

In [ ]:
def get_rooms_points(buildings):
    """maps building ids to their points and rooms,
    ie {4:{'rooms':{5}}}"""
    result = {}
    for building_id, name in buildings.items():
        building_data = {
            'rooms': map_rooms(api.building_rooms(building_id)),
            'points': map_points(api.building_points(building_id))
        }
        result[building_id] = building_data
    return result

def map_points(points):
    """Take a pandas df returned from building_points
    and turn it into something easier to use on the frontend
    """
    results = {}
    for index, row in points.iterrows():
        results[row['id']] = row['name'] + '- ' + row['description']
    return results


def map_rooms(rooms):
    """Similar to the map_points function but for rooms"""
    results = {}
    for index, row in rooms.iterrows():
        results[row['id']] = row['name'].replace("_", " ")
    return results